In [31]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import trimesh as tm
import boolean_marching_cubes as bmc

In [32]:
def indexed(l, start=0):
    return list(enumerate(l, start))

In [33]:
l_bis = []

templates_path = os.path.relpath('../data/bmc/lattice')

for i in range(2**8):
    bi = np.array(list(np.binary_repr(i, width=8))).astype(int).reshape((2, 2, 2))
    l_bi = tg.to_lattice(bi, [0, 0, 0])
    l_bis.append(l_bi)
    e_path = os.path.join(templates_path, 'l_bis_' + f'{i:03}' + '.csv')
    bmc.to_csv(l_bi, e_path)

## Create 256 empty configurations

In [34]:
# set tileset number / window, door, wall etc
numtile = 1 
numbertileset = 255*numtile
tileset = [] 
for i in range (2**8):
    c_id = i + numbertileset
    tileset.append(c_id)


In [35]:
# creating empty list of tilesets 

empty_tiles_path = os.path.relpath('../data/bmc/newsaving')
empty_tiles_meshes = []

for i in range(2**8):
    empty_tile = tm.creation.icosphere(subdivisions=1, radius=0.01)
    empty_tiles_meshes.append(empty_tile)

def emp_save_tile_meshes(empty_tiles_meshes, l_bis, tiles_path):

    for tile_mesh, l_bis in zip(empty_tiles_meshes, l_bis):

        for i in range (2**8):
            c_id = i + numbertileset

        # saving mesh
            tile_mesh_path = os.path.join(tiles_path, 't_' + f'{c_id:04}' + '.obj')
            with open(tile_mesh_path, 'w') as file:
                file.write(tm.exchange.obj.export_obj(tile_mesh))
            
        return tile_mesh

emp_save_tile_meshes(empty_tiles_meshes,l_bis,empty_tiles_path)



<trimesh.Trimesh(vertices.shape=(42, 3), faces.shape=(80, 3))>

In [36]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

p = pv.Plotter(notebook=False)

base_lattice = l_bis[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5 
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit *0.5

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    i = int(value)
    lattice = l_bis[i]

    # Add the data values to the cell data
    grid.cell_arrays["cube"] = lattice.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="cube")
    # adding the voxels
    p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=0.4, show_scalar_bar=False, color="C97064")

    # adding the meshes
    p.add_mesh(tri_to_pv(mesh), color='#abd8ff', name="sphere")

    return

p.add_slider_widget(create_mesh, [0, len(l_bis)+1], title='Tiles', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))


    
p.show(use_ipyvtk=True)

  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_26532/1824185372.py", line 38, in create_mesh
    p.add_mesh(tri_to_pv(mesh), color='#abd8ff', name="sphere")
NameError: name 'mesh' is not defined


[(1.9318516525781368, 1.9318516525781368, 1.9318516525781368),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 1.0)]